<a href="https://colab.research.google.com/github/geraldmc/DL4CV-2022/blob/main/test_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Standard libraries
import os
import math
import numpy as np 
import time

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline 
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgba
import seaborn as sns
sns.set()

## Progress bar
from tqdm.notebook import tqdm

In [2]:
import torch
print("Using torch", torch.__version__)

# Note on Mac M1 this prints ... 'Using torch 1.9.1.post3'

Using torch 1.10.0+cu111
